In [2]:
import json, time, re, requests, pickle
import pandas as pd
import psycopg2 as pg2
import numpy as np

from sqlalchemy import create_engine
from psycopg2.extras import RealDictCursor, Json
from psycopg2.extensions import AsIs

%matplotlib inline
%run ../assets/sql_cred.py

In [3]:
def filename_format_log(file_path, 
                        logfile = '../assets/file_log.txt', 
                        now = round(time.time()), 
                        file_description = None): 
   
    try:
        ext = re.search('(?<!^)(?<!\.)\.(?!\.)', file_path).start() 
    except:
        raise NameError('Please enter a relative path with a file extension.') 
    
    stamp = re.search('(?<!^)(?<!\.)[a-z]+_[0-z]+(?=\.)', file_path).start()
    formatted_name = f'{file_path[:stamp]}{now}_{file_path[stamp:]}'  
    if not file_description:
        file_description = f'Saved at: {time.asctime(time.gmtime(now))}'
    with open(logfile, 'a+') as f:
        f.write(f'{formatted_name}: {file_description}\n')
    return formatted_name, now, file_description

In [4]:
engine = create_engine(ENGINE)

In [5]:
def con_cur_to_db(dbname=DBNAME, dict_cur=None):
    con = pg2.connect(host=IP_ADDRESS,
                  dbname=dbname,
                  user=USER,
                  password=PASSWORD)
    if dict_cur:
        cur = con.cursor(cursor_factory=RealDictCursor)
    else:
        cur = con.cursor()
    return con, cur
    
def execute_query(query, dbname=DBNAME, dict_cur=None, command=False):
    con, cur = con_cur_to_db(dbname, dict_cur)
    cur.execute(f'{query}')
    if not command:
        data = cur.fetchall()
        con.close()
        return data
    con.commit() #sends to server
    con.close() #closes server connection

In [6]:
query = '''SELECT * FROM track_list;'''
response = execute_query(query, dict_cur=True)
track_df = pd.DataFrame(response)
track_df.set_index('track_id', inplace=True)
track_df.head()

,album_name,artist_name,lyrics,playlist_id,playlist_name,playlist_owner,track_name
track_id,,,,,,,
0h7TlF8gKb61aSm874s3cV,I Can't Tell You How Much It Hurts,moow,\n\nIf your needle is near\nNeedle is near\nYo...,37i9dQZF1DXarebqD2nAVg,Tender,spotify,You'r in My Head
6koowTu9pFHPEcZnACLKbK,Coming Home,Leon Bridges,\n\n[Verse 1]\nBrown skin girl on the other si...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Brown Skin Girl
1JkhKUXAoNivi87ipmV3rp,Back To Love (Deluxe Version),Anthony Hamilton,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Best of Me
51lPx6ZCSalL2kvSrDUyJc,The Search for Everything,John Mayer,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,You're Gonna Live Forever in Me
3vqlZUIT3rEmLaYKDBfb4Q,Songs In The Key Of Life,Stevie Wonder,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,37i9dQZF1DX4adj7PFEBwf,Wedding Bells,spotify,Isn't She Lovely


In [164]:
lyric_df = track_df[['lyrics']]

In [167]:
lyric_df = lyric_df.dropna(axis=0)

In [168]:
lyric_df.shape

(2276, 1)

In [169]:
lyric_df.head()

,lyrics
track_id,
0h7TlF8gKb61aSm874s3cV,\n\nIf your needle is near\nNeedle is near\nYo...
6koowTu9pFHPEcZnACLKbK,\n\n[Verse 1]\nBrown skin girl on the other si...
1JkhKUXAoNivi87ipmV3rp,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ..."
51lPx6ZCSalL2kvSrDUyJc,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...
3vqlZUIT3rEmLaYKDBfb4Q,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...


In [170]:
lyric_df.loc['0h7TlF8gKb61aSm874s3cV', 'lyrics']

'\n\nIf your needle is near\nNeedle is near\nYou can take my blood\nOh I saved it for you\nHundreds of drops\nRunning red\n\nNeedle is near\nYou can take my blood\nOh I saved it for you\nHundreds of drops\nRunning red\nNeedle is near\nOh I saved it for you\nHundreds of drops\nRunning red\n\n'

In [44]:
def clean_lyrics(lyrics, keep_tags=False, keep_nl=False):

    text = lyrics.lower()

    if not keep_tags:
        text = re.sub('(\[.+\])', '', text)

    text = re.sub('[^a-z\s]', '', text)

    text = text.strip()
    text = re.sub('\n\n', ' ', text) 

    text = text.strip()

    if keep_nl:
        text = re.sub('\n', ' \n ', text)
    else:
        text = re.sub('\n', ' ', text)


    return text

In [172]:
lyric_df['clean_lyrics'] = lyric_df.lyrics.map(lambda x: clean_lyrics(x, keep_tags=False, keep_nl=True))

In [173]:
lyric_df.head()

,lyrics,clean_lyrics
track_id,,
0h7TlF8gKb61aSm874s3cV,\n\nIf your needle is near\nNeedle is near\nYo...,if your needle is near \n needle is near \n yo...
6koowTu9pFHPEcZnACLKbK,\n\n[Verse 1]\nBrown skin girl on the other si...,brown skin girl on the other side of the room ...
1JkhKUXAoNivi87ipmV3rp,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",its simple i love it \n having you near me hav...
51lPx6ZCSalL2kvSrDUyJc,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,a great big bang and dinosaurs \n fiery rainin...
3vqlZUIT3rEmLaYKDBfb4Q,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,isnt she lovely \n isnt she wonderful \n isnt ...


In [174]:
not_eng = lyric_df[lyric_df['clean_lyrics'].str.contains('(\sdel\s)|(\sque\s)|(\svous\s)|(\setre\s)')].index.tolist()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [175]:
lyric_df = lyric_df.drop(index=not_eng, axis=0)

In [176]:
not_lyrics = lyric_df[lyric_df['clean_lyrics'].str.contains('lyrics for this song have yet')].index.tolist()

In [177]:
lyric_df = lyric_df.drop(index=not_lyrics, axis=0)

In [178]:
long_gap = lyric_df[lyric_df['clean_lyrics'].str.contains(r'(\s{10,})')].index.tolist()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.


In [179]:
lyric_df = lyric_df.drop(index=long_gap, axis=0)

In [ ]:
lyric_df = lyric_df.drop(index=1768, axis=0)

In [180]:
lyric_df.head()

,lyrics,clean_lyrics
track_id,,
0h7TlF8gKb61aSm874s3cV,\n\nIf your needle is near\nNeedle is near\nYo...,if your needle is near \n needle is near \n yo...
6koowTu9pFHPEcZnACLKbK,\n\n[Verse 1]\nBrown skin girl on the other si...,brown skin girl on the other side of the room ...
1JkhKUXAoNivi87ipmV3rp,"\n\n[Verse 1]\nIt's simple, I love it\nHaving ...",its simple i love it \n having you near me hav...
51lPx6ZCSalL2kvSrDUyJc,\n\n[Intro: Whistling]\n\n[Verse 1]\nA great b...,a great big bang and dinosaurs \n fiery rainin...
3vqlZUIT3rEmLaYKDBfb4Q,\n\n[Verse 1]\nIsn't she lovely\nIsn't she won...,isnt she lovely \n isnt she wonderful \n isnt ...


In [181]:
lyric_df.describe()

,lyrics,clean_lyrics
count,1921,1921
unique,1806,1801
top,\n\n[Verse 1]\nAll I want is nothing more\nTo ...,
freq,4,6


In [182]:
lyric_df = lyric_df.drop_duplicates(subset=['clean_lyrics'])

In [183]:
lyric_df.describe()

,lyrics,clean_lyrics
count,1801,1801
unique,1801,1801
top,\n\n[Verse 1]\nHow fickle my heart and how woo...,when the rain is blowing in your face \n and t...
freq,1,1


In [ ]:
# formatted_name, now, file_description= filename_format_log(file_path = '../assets/clean_lyrics.csv')

# lyric_df.to_csv(formatted_name, index=False)

In [68]:
eda_df = pd.DataFrame(columns=['id', 'line', 'line_num', 'tag'])

for idx, track in lyric_df[['lyrics']].iterrows():
    track = re.sub('(\[[^\]]+\]\\n\\n)', '', track[0]) #removes tags without lyrics
    track = re.sub('\[[^\[]+(:([^\]]+))\]', '', track) #removes all text after :
    track = re.sub('(^|)(\[Ad-Libs\])|(\[Keisha\])|(\[Shane Powers\])', '', track) #removes artist and non-standard tags
    track = re.sub('(^|)(\[Break\])|(\[Vocals / Lyrics By\])|(\[Skit\])', '', track) #removes artist and non-standard tags
    
    tags = re.findall('(\[[^\]]+\])', track) #finds all tags
    track = re.sub('(\[[^\]]+\])', '', track) #removes all tags
              
    track = track.strip().split('\n\n') 
    track_split = [verse.strip().split('\n') for verse in track] 
    
    v_count = len(track_split)

    for i in range(v_count):
        if len(tags) == v_count:
            tag = tags[i]
        else:
            tag = f'[Verse {(i+1)}]'
        
        for j, line in enumerate(track_split[i]):
            index = 1
            
            row = {
            'id': idx,
            'line':line,
            'line_num':(j+1),
            'tag': tag
            }
            
            eda_df = eda_df.append(row, ignore_index=True)

In [71]:
eda_df

,id,line,line_num,tag
0,0h7TlF8gKb61aSm874s3cV,If your needle is near,1,[Verse 1]
1,0h7TlF8gKb61aSm874s3cV,Needle is near,2,[Verse 1]
2,0h7TlF8gKb61aSm874s3cV,You can take my blood,3,[Verse 1]
3,0h7TlF8gKb61aSm874s3cV,Oh I saved it for you,4,[Verse 1]
4,0h7TlF8gKb61aSm874s3cV,Hundreds of drops,5,[Verse 1]
5,0h7TlF8gKb61aSm874s3cV,Running red,6,[Verse 1]
6,0h7TlF8gKb61aSm874s3cV,Needle is near,1,[Verse 2]
7,0h7TlF8gKb61aSm874s3cV,You can take my blood,2,[Verse 2]
8,0h7TlF8gKb61aSm874s3cV,Oh I saved it for you,3,[Verse 2]
9,0h7TlF8gKb61aSm874s3cV,Hundreds of drops,4,[Verse 2]


In [72]:
# formatted_name, now, file_description = filename_format_log(file_path ='../assets/eda_df.csv')
# eda_df.to_csv(formatted_name, index=False)

In [191]:
eda_df = pd.read_csv('../assets/1548892595_eda_df.csv')

In [192]:
eda_df.tag.value_counts()[eda_df.tag.value_counts() < 10]

[Jazz]                          9
[Verse 24]                      9
[Chorus/Outro]                  9
[verse]                         9
[Rod Stewart]                   9
[Verse  4]                      9
[Brian Morgan]                  8
[Sisqo]                         8
[Coko]                          8
[Verse 1 – Johnny Drille]       8
[Mike]                          8
[Verse 2 – Simi]                8
[Modified Chorus]               8
[VERSE 1]                       8
[Intro Skit]                    7
[Verse 2 / Outro]               7
[Verse 26]                      7
[Hook 1]                        7
[Chorus 3/Outro]                7
[Verse  2]                      7
[Verso 2]                       6
[outro]                         6
[Chorus (repeat until fade)]    6
[Verse 1 (reprise)]             6
[bridge]                        6
[Verso 3]                       6
[Verse 27]                      6
[Vesre 1]                       6
[Instrumental]                  5
[Verso 1]     

In [193]:
eda_df[eda_df.loc[:,'tag'] == '[Verse 94]']

,id,line,line_num,tag
9880,4TYZXfu6VeblQMK2TwbDte,GEORGE,1,[Verse 94]
9881,4TYZXfu6VeblQMK2TwbDte,Did you get a doctor?,2,[Verse 94]


In [194]:
track_df.loc['4TYZXfu6VeblQMK2TwbDte']

album_name                                    It's a Wonderful Life
artist_name                                                 S E S H
lyrics            \n\nEXT. TREE-LINED RESIDENTIAL STREET – NIGHT...
playlist_id                                  37i9dQZF1DXbtuVQL4zoey
playlist_name                                            Cute Beats
playlist_owner                                              spotify
track_name                                    It's a Wonderful Life
Name: 4TYZXfu6VeblQMK2TwbDte, dtype: object

In [195]:
drop = eda_df[eda_df['id'] == '4TYZXfu6VeblQMK2TwbDte'].index.tolist()

In [196]:
eda_df = eda_df.drop(drop, axis=0)

In [197]:
lyric_df = lyric_df.drop(index='4TYZXfu6VeblQMK2TwbDte', axis=0)

In [198]:
eda_df.tag.value_counts()[eda_df.tag.value_counts() < 10]

[Rod Stewart]                   9
[verse]                         9
[Jazz]                          9
[Chorus/Outro]                  9
[Verse  4]                      9
[Brian Morgan]                  8
[Coko]                          8
[Mike]                          8
[Verse 1 – Johnny Drille]       8
[Verse 2 – Simi]                8
[VERSE 1]                       8
[Sisqo]                         8
[Modified Chorus]               8
[Chorus 3/Outro]                7
[Verse  2]                      7
[Hook 1]                        7
[Verse 24]                      7
[Verse 2 / Outro]               7
[Intro Skit]                    7
[outro]                         6
[Verso 2]                       6
[Vesre 1]                       6
[Verse 1 (reprise)]             6
[bridge]                        6
[Chorus (repeat until fade)]    6
[Verso 3]                       6
[Verso 1]                       5
[Verse 27]                      5
[Verse 26]                      5
[Bridge 2 (2x)

In [199]:
eda_df[eda_df.loc[:,'tag'] == '[Verse 55]']

,id,line,line_num,tag
2990,6xYd4zCVeSp80Un2Rl9wDs,Cruisin' is made for love,1,[Verse 55]


In [200]:
lyric_df.loc['6xYd4zCVeSp80Un2Rl9wDs',:]['lyrics']

"\n\nBaby let's cruise away from here\n\nDon't be confused baby, the way is clear\n\nAnd if you want it you got it forever\n\nOh, this is not a one night stand baby\n\nLet the music take your mind\n\nAnd just release and you will find, baby\n\nWe're going to fly away\n\nGlad you're going my way\n\nI love it when we're cruisin' together\n\nMusic was made for love\n\nCruisin' is made for love\n\nI love it when we're cruisin' together\n\nBaby, tonight belongs to us\n\nEverything's right, do what you must, baby\n\nAnd inch by inch we get closer and closer\n\nEvery little part of each other ooh, baby, baby\n\nLet the music take your mind\n\nJust release and you will find, baby\n\nWe're going to fly away\n\nGlad you're going my way\n\nI love it when we're cruisin' together\n\nMusic was made for love\n\nCruisin' is made for love\n\nI love you when we're cruisin' together\n\nWe're going to fly away\n\nGlad you're going my way\n\nI love it when we're cruisin' together\n\nMusic was made for love

In [143]:
off_verse = eda_df[eda_df.id =='6xYd4zCVeSp80Un2Rl9wDs'].index.tolist()

In [202]:
eda_df.tag.loc[off_verse] = eda_df.tag.loc[off_verse].map(lambda x: '[Verse 1]')

In [203]:
eda_df.tag.loc[off_verse]

2936    [Verse 1]
2937    [Verse 1]
2938    [Verse 1]
2939    [Verse 1]
2940    [Verse 1]
2941    [Verse 1]
2942    [Verse 1]
2943    [Verse 1]
2944    [Verse 1]
2945    [Verse 1]
2946    [Verse 1]
2947    [Verse 1]
2948    [Verse 1]
2949    [Verse 1]
2950    [Verse 1]
2951    [Verse 1]
2952    [Verse 1]
2953    [Verse 1]
2954    [Verse 1]
2955    [Verse 1]
2956    [Verse 1]
2957    [Verse 1]
2958    [Verse 1]
2959    [Verse 1]
2960    [Verse 1]
2961    [Verse 1]
2962    [Verse 1]
2963    [Verse 1]
2964    [Verse 1]
2965    [Verse 1]
          ...    
2968    [Verse 1]
2969    [Verse 1]
2970    [Verse 1]
2971    [Verse 1]
2972    [Verse 1]
2973    [Verse 1]
2974    [Verse 1]
2975    [Verse 1]
2976    [Verse 1]
2977    [Verse 1]
2978    [Verse 1]
2979    [Verse 1]
2980    [Verse 1]
2981    [Verse 1]
2982    [Verse 1]
2983    [Verse 1]
2984    [Verse 1]
2985    [Verse 1]
2986    [Verse 1]
2987    [Verse 1]
2988    [Verse 1]
2989    [Verse 1]
2990    [Verse 1]
2991    [Verse 1]
2992    [V

In [ ]:
for i in range(len(verses[0])):
    words = verses[0][i].split()
    print(words)

In [ ]:
# lyric_df[['verses']] = json.dumps(verses)
# eval(foo) #alternative for json.reads()